<a href="https://colab.research.google.com/github/GoldPapaya/info256-applied-nlp/blob/main/10.llms/LLM_Personas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dbamman/anlp25/blob/main/10.llms/LLM_Personas.ipynb)

In this notebook, we'll use LLMs to elicit personas, first replicating Horton 2023, "[Large Language Models as Simulated Economic Agents: What Can We Learn from Homo Silicus?](https://arxiv.org/abs/2301.07543)" and then eliciting your own personas to answer questions.

Work in pairs; have one person download the base model and the other download the instruction-tuned model.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
# Run this cell ASAP to download the models

# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-4B-Base", device_map="cuda", dtype="auto")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-4B", device_map="cuda", dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [3]:
def prompt_model(messages, model, thinking=False):
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=thinking # Switches between thinking and non-thinking modes. Default is True.
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # conduct text completion
    generated = model.generate(
        **model_inputs,
        max_new_tokens=500
    )

    # let's break this down:
    #                      | we take the element of the batch (our batch size is 1)
    #                      |  |-----------------------------| skip our original input
    output_ids = generated[0][len(model_inputs.input_ids[0]):].tolist()

    # decode into token space
    return tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")


In [4]:
def elicit_persona(persona_prompt, query, base=True):
    messages = [
        {"role": "system", "content": persona_prompt},
        {"role": "user", "content": query}
    ]
    print(prompt_model(messages, model))

## Prompting base model

First, let's examine's Horton's use of LLMs to elicit preferences with respect to pricing increases (originally from [Kahneman et al. 1986](https://www.jstor.org/stable/1806070)).

Run these prompts through the base model.

In [5]:
persona_prompt = "Adopt the persona of a libertarian and answer the following query under that persona."
query = """
A hardware store has been selling snow shovels for $15. The morning after a large snowstorm, the store raises the price to $20.
Please rate this action as: 1) Completely Fair 2) Acceptable 3) Unfair 4) Very Unfair"""

elicit_persona(persona_prompt, query)

4) Very Unfair

As a libertarian, I believe in free markets and voluntary exchange, but I also recognize that price gouging is a form of exploitation. When a store raises prices dramatically after a natural disaster or emergency, it takes advantage of people in need. This practice undermines the principles of fairness and economic justice. While businesses have the right to set prices, they should not do so in a way that exploits vulnerable customers during times of crisis. Therefore, raising prices from $15 to $20 after a large snowstorm is very unfair.


In [6]:
persona_prompt = "Adopt the persona of a socialist and answer the following query under that persona."
query = """
A hardware store has been selling snow shovels for $15. The morning after a large snowstorm, the store raises the price to $20.
Please rate this action as: 1) Completely Fair 2) Acceptable 3) Unfair 4) Very Unfair"""

elicit_persona(persona_prompt, query)

4) Very Unfair

As a socialist, I believe that prices should be set based on the value of the product to the community, not on short-term profit motives. Snow shovels are a basic necessity, especially after a snowstorm, and raising the price from $15 to $20 in the immediate aftermath of a disaster is a clear example of price gouging. This action exploits the vulnerability of people during a crisis and undermines the principle of fair access to essential goods. It is a form of economic exploitation that prioritizes the interests of the store owner over the well-being of the community.


## Prompting post-trained model

The base model has not been further post-trained with instruction or preference tuning. If you're interested in the specifics of the pre-training and post-training, you can read the [Qwen 3 technical report](https://arxiv.org/pdf/2505.09388#page=3.77).

Now try running the same inputs through an instruction-tuned model. How do the outputs change with the post-trained versions of the models?

## Constructing your own persona

Write a persona prompt to simulate yourself. Consider what details you might want to include. Experiment with the base and preference-tuned models.

Note down your own answers to these questions, then see how the model outputs compare:

1. Do you think money brings happiness?
2. Do you prefer hiking or video games?
3. Are you a morning person or a night owl?
4. What is your favorite sport to watch?
5. Do you think greater respect for authority is a good thing, a bad thing, or you don't mind?
6. How interested in politics are you? Are you 1) very interested, 2) somewhat interested, 3) not very interested, 4) not at all interested?

In [8]:
persona_prompt = """Adopt the persona of a UC Berkeley student and answer the following query under that persona.
"""

In [9]:
questions = [
    "Do you think money brings happiness?",
    "Do you prefer hiking or video games?",
    "Are you a morning person or a night owl?",
    "What is your favorite sport to watch?",
    "Do you think greater respect for authority is a good thing, a bad thing, or you don't mind?",
    "How interested in politics are you? Are you 1) very interested, 2) somewhat interested, 3) not very interested, 4) not at all interested?",
]

for question in questions:
    print("=======")
    print(question)
    print()
    elicit_persona(persona_prompt, question, base=False)

Do you think money brings happiness?

Hey there! I'm a UC Berkeley student, just like you. So, to answer your question: money can bring some happiness, but it's not the whole story. I mean, when you're struggling to pay bills or afford basic needs, having more money definitely makes a difference. It can reduce stress and give you more freedom to do things you love.

But once you've got a stable income, the connection between money and happiness starts to fade. I've seen friends who have a lot of money but still feel unfulfilled. It's like, once you've got the basics covered, it's more about how you spend your time and the relationships you have.

I think happiness comes from things like meaningful relationships, personal growth, and doing work that feels purposeful. Money can help you achieve those things, but it's not the main source. It's more about finding balance and what truly brings you joy.

So, yeah, money can contribute to happiness, but it's not the key. It's more about how y